### NumPy
NumPy: a multidimensional array object.  
The rationale behind NumPy is the following: Python being a high-level dynamic language, it is easier to use but slower than a low-level language such as C. NumPy implements the multidimensional array structure in C and provides a convenient Python interface, thus bringing together high performance and ease of use. NumPy is used by many Python libraries. For example, pandas is built on top of NumPy.  

ndarray: n-dimensional arrays of homogeneous data types  
-- fixed size at creation  
-- same data type  

attributes of an ndarray object are:  
-- ndarray.ndim：the number of axes (dimensions) of the array.  
-- ndarray.shape：the dimensions of the array.   
-- ndarray.size：the total number of elements of the array.   
-- ndarray.dtype：an object describing the type of the elements in the array.   
-- ndarray.itemsize：the size in bytes of each element of the array.   
-- ndarray.data：the buffer containing the actual elements of the array.   

`np.set_printoptions(threshold=sys.maxsize)`

`python3 -c "import numpy; numpy.info(numpy.add)"`

In [2]:
import random
import numpy as np

In [14]:
# python 对比 numpy
# 1. In NumPy, array operations are implemented internally with C loops rather than Python loops. 
#    Python is typically slower than C because of its interpreted and dynamically-typed nature.
# 2. The data in a NumPy array is stored in a contiguous block of memory in RAM. 
#    This property leads to more efficient use of CPU cycles and cache.

n = 1000000
x = [random.random() for _ in range(n)]
y = [random.random() for _ in range(n)]
print(x[:3])
print(y[:3])
z = [x[i] + y[i] for i in range(n)]
print(z[:3])
%timeit [x[i] + y[i] for i in range(n)]

xa = np.array(x)
ya = np.array(y)
za = xa + ya
print(za[:3])
%timeit xa + ya

%timeit sum(x)
%timeit np.sum(xa)

d = [abs(x[i] - y[j])
     for i in range(1000)
     for j in range(1000)]
# np.newaxis, add a new axis to broadcasting 
da = np.abs(xa[:1000, np.newaxis] - ya[:1000])

[0.211522339052433, 0.6463004828834406, 0.33267705861101304]
[0.24731978603035942, 0.5552296500719227, 0.9454938018764355]
[0.45884212508279243, 1.2015301329553632, 1.2781708604874487]
254 ms ± 31.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
[0.45884213 1.20153013 1.27817086]
3.97 ms ± 454 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.13 ms ± 599 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
642 µs ± 10.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Element-wise : the same shape.  
Broadcasting : different shapes.

In [6]:
a = np.array([[1,2,3], [4,5,6]])
print(a)
print(a.ndim)
print(a.shape)
print(a.sum(axis=0), a.sum(axis=1))

[[1 2 3]
 [4 5 6]]
2
(2, 3)
[5 7 9] [ 6 15]


In [15]:
b = np.array([[[1,2,3], [4,5,6]],[[11,12,13],[14,15,16]]])
print(b)
print(b.ndim)
print(b.shape)
print(b.sum(axis=0))
print(b.sum(axis=1))
print(b.sum(axis=2))

A = np.array([1,2,3])
B = np.array([4,5,6])
C = np.array([11,12,13])
D = np.array([14,15,16])
new_b = np.array([[A, B], [C, D]])
print(new_b)

[[[ 1  2  3]
  [ 4  5  6]]

 [[11 12 13]
  [14 15 16]]]
3
(2, 2, 3)
[[12 14 16]
 [18 20 22]]
[[ 5  7  9]
 [25 27 29]]
[[ 6 15]
 [36 45]]
[[[ 1  2  3]
  [ 4  5  6]]

 [[11 12 13]
  [14 15 16]]]


### Single Instruction, Multiple Data, SIMD

In [1]:
import numpy as np

def aid(x):
    # This function returns the memory block address of an array
    return x.__array_interface__['data'][0]

a = np.zeros(3)
aid(a), aid(a[1:])

(140447628974448, 140447628974456)

In [3]:
def get_data_base(arr):
    """For a given NumPy array, find the base array that owns the actual data."""
    base = arr
    while isinstance(base.base, np.ndarray):
        base = base.base
    return base

def arrays_share_data(x, y):
    return get_data_base(x) is get_data_base(y)

print(arrays_share_data(a, a.copy()))
print(arrays_share_data(a, a[:1]))

False
True


In [4]:
import numpy as np
a = np.zeros(10)
ax = aid(a)
ax

140447593509008

In [5]:
b = a.copy() # copy() 浅复制
aid(b) == ax

False

In [6]:
a *= 2 # in-place operations
aid(a) == ax

True

In [7]:
c = a * 2 # implicit-copy operations
aid(c) == ax

False

In [8]:
%%timeit a = np.zeros(10000000)
a *= 2

10.9 ms ± 599 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%%timeit a = np.zeros(10000000) # implicit-copy operations are slower
b = a * 2

24.6 ms ± 274 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
a = np.zeros((100, 100))
ax = aid(a)

b = a.reshape((1, -1))
aid(b) == ax

True

In [12]:
# Reshaping an array may or may not involve a copy. 
# The reasons will be explained in the How it works... section. 
# For instance, reshaping a 2D matrix does not involve a copy, 
# unless it is transposed (or more generally, non-contiguous):

c = a.T.reshape((1, -1)) 
aid(c) == ax

False

In [13]:
%timeit b = a.reshape((1, -1))

557 ns ± 36.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [14]:
%timeit a.T.reshape((1, -1))

9.32 µs ± 523 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [15]:
# the flatten() method always returns a copy, and the ravel() method returns a copy only if necessary 
# (thus it's faster, especially with large arrays).

d = a.flatten()
aid(d) == ax

False

In [16]:
e = a.ravel()
aid(e) == ax

True

In [17]:
%timeit a.flatten()

6.07 µs ± 475 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [18]:
%timeit a.ravel()

339 ns ± 16.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [19]:
# Broadcasting rules allow us to make computations on arrays with different but compatible shapes. 
# In other words, we don't always need to reshape or tile our arrays to make their shapes match. 
# The following example illustrates two ways of doing an outer product between two vectors: 
# the first method involves array tiling, the second one (faster) involves broadcasting:

n = 1000

a = np.arange(n)
ac = a[:, np.newaxis]  # column vector
ar = a[np.newaxis, :]  # row vector

In [20]:
%timeit np.tile(ac, (1, n)) * np.tile(ar, (n, 1))

16.9 ms ± 2.24 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
%timeit ar * ac

3.48 ms ± 330 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Why are NumPy arrays efficient

A NumPy array is basically described by metadata (notably the number of dimensions, the shape, and the data type) 
and the actual data. The data is stored in a homogeneous and contiguous block of memory, 
at a particular address in system memory (Random Access Memory, or RAM). 
This block of memory is called the data buffer. 
This is the main difference between an array and a pure Python structure, 
such as a list, where the items are scattered across the system memory. 
This aspect is the critical feature that makes NumPy arrays so efficient.

Why is this so important? Here are the main reasons:

1. Computations on arrays can be written very efficiently in a low-level language such as C 
    (and a large part of NumPy is actually written in C). Knowing the address of the memory block and the data type, 
    it is just simple arithmetic to loop over all items, for example. There would be a significant overhead to do 
    that in Python with a list.
2. Spatial locality in memory access patterns results in performance gains notably due to the CPU cache. 
    Indeed, the cache loads bytes in chunks from RAM to the CPU registers. Adjacent items are then loaded very 
    efficiently (sequential locality, or locality of reference).
3. Finally, the fact that items are stored contiguously in memory allows NumPy to take advantage of vectorized 
    instructions of modern CPUs, such as Intel's SSE and AVX, AMD's XOP, and so on. For example, multiple 
    consecutive floating point numbers can be loaded in 128, 256, or 512 bits registers for vectorized arithmetical 
    computations implemented as CPU instructions.

Additionally, NumPy can be linked to highly optimized linear algebra libraries such as BLAS and LAPACK through ATLAS 
or the Intel Math Kernel Library (MKL). A few specific matrix computations may also be multithreaded, 
taking advantage of the power of modern multicore processors.

In conclusion, storing data in a contiguous block of memory ensures that the architecture of modern CPUs 
is used optimally, in terms of memory access patterns, CPU cache, and vectorized instructions.

## Why can't some arrays be reshaped without a copy?

![image](./layout.png)

NumPy uses the row-major order, like C, but unlike FORTRAN.
More generally, NumPy uses the notion of strides to convert between a multidimensional index and the memory location of the underlying (1D) sequence of elements. 
The specific mapping between array[i1, i2] and the relevant byte address of the internal data is given by:
`offset = array.strides[0] * i1 + array.strides[1] * i2`

When reshaping an array, NumPy avoids copies when possible by modifying the strides attribute.

In [22]:
x = np.zeros(10)
x.strides

(8,)

In [23]:
y = np.zeros((10, 10))
y.strides

(80, 8)

In [24]:
n = 1000
a = np.arange(n)

In [25]:
b = np.lib.stride_tricks.as_strided(a, (n, n), (0, 8))

In [26]:
b.size, b.shape, b.nbytes

(1000000, (1000, 1000), 8000000)

In [27]:
%timeit b * b.T

3.38 ms ± 454 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
%%timeit
np.tile(a, (n, 1)) * np.tile(a[:, np.newaxis], (1, n))

11.9 ms ± 1.79 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
https://github.com/ipython-books/cookbook-2nd chp6